In [1]:
# Install the Together API package
!pip install together

In [2]:
import os
import pandas as pd
import together
import logging
import time
import pickle

In [3]:
os.environ["TOGETHER_API_KEY"] = "4bd67bf8d915ebea6c3d8a07bc5d4eac77fdc9846e2068940980fdb5eff5d0ac"
print("TOGETHER_API_KEY:", os.getenv("TOGETHER_API_KEY"))

TOGETHER_API_KEY: 4bd67bf8d915ebea6c3d8a07bc5d4eac77fdc9846e2068940980fdb5eff5d0ac


In [4]:
class TogetherLLM:
    """Together large language models with retry mechanism."""
    def __init__(self, model: str, api_key: str, temperature: float = 0.7, max_tokens: int = 120000, retries: int = 3, backoff_factor: float = 1.5):
        self.model = model
        self.api_key = api_key
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.retries = retries
        self.backoff_factor = backoff_factor
        together.api_key = api_key  # Set the API key directly

    def call(self, prompt: str) -> str:
        """Call to Together endpoint with retry mechanism."""
        attempt = 0
        while attempt < self.retries:
            try:
                response = together.Completion.create(
                    model=self.model,
                    prompt=prompt,
                    max_tokens=self.max_tokens,
                    temperature=self.temperature
                )
                # Return response if successful
                return response.choices[0].text.strip()
            except (together.error.APIError, together.error.APIConnectionError, together.error.InvalidRequestError) as e:
                logging.warning(f"Attempt {attempt + 1} failed: {e}")
                attempt += 1
                time.sleep(self.backoff_factor ** attempt)  # Exponential backoff

        # If all retries fail, raise an exception
        raise Exception(f"Failed to complete after {self.retries} retries")


In [5]:
def load_pickle_file(file_path: str):
    """Load the pickle file into a DataFrame."""
    with open(file_path, 'rb') as file:
        return pickle.load(file)

In [6]:
def process_chunk_sequentially(chunk: pd.DataFrame, model: TogetherLLM, previous_summary: str = "") -> str:
    """Process a chunk sequentially and return the biography section."""
    # Ensure that all values are strings
    chunk['Transkript'] = chunk['Transkript'].astype(str)

    # Combine all rows of the chunk into a single text
    chunk_text = " ".join(chunk['Transkript'].tolist())

    # Step 1: Create a prompt that considers whether there is a previous summary
    if previous_summary:
        prompt = (
            "Hier ist die bisher erstellte Biografie:\n{previous_summary}\n\n"
            "Bitte fassen Sie die folgenden Interviewinformationen zusammen und konzentrieren Sie sich auf wichtige Lebensereignisse, "
            "insbesondere auf die erwähnten Jahre und biografisch bedeutsame Ereignisse. Vermeiden Sie Wiederholungen und irrelevante Details. "
            "Integrieren Sie neue relevante Informationen in die bestehende Biografie.\n\n{chunk_text}"
        )
    else:
        prompt = (
            "Erstellen Sie eine vollständige und detaillierte Biografie basierend auf den folgenden Interviewinformationen. "
            "Konzentrieren Sie sich dabei auf wichtige Lebensereignisse und die in den Interviews erwähnten Jahre. "
            "Vermeiden Sie Wiederholungen und irrelevante Details.\n\n{chunk_text}"
        )

    # Print the prompt that will be sent to the model
    print("\n==== Nächste Eingabeaufforderung an das Modell ====\n")
    print(prompt.format(previous_summary=previous_summary, chunk_text=chunk_text))

    # Call the model with the constructed prompt
    biography_section = model.call(prompt.format(previous_summary=previous_summary, chunk_text=chunk_text))

    # Print the intermediate summary returned by the model
    print("\n==== Zwischenergebnis (Zusammenfassung des Abschnitts) ====\n")
    print(biography_section)

    return biography_section


In [8]:
def main(pickle_file_path: str):
    # Load the pickle file
    df = load_pickle_file(pickle_file_path)

    # Initialize your model
    model = TogetherLLM(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        api_key=os.getenv("TOGETHER_API_KEY"),
        temperature=0.7,
        max_tokens=120000,
        retries=5,  # Increased retries for stability
        backoff_factor=1.5  # Exponential backoff factor
    )

    # Process each chunk sequentially
    chunk_size = 250
    previous_summary = ""

    for start in range(0, len(df), chunk_size):
        chunk = df.iloc[start:start + chunk_size]
        try:
            # Process each chunk sequentially
            previous_summary = process_chunk_sequentially(chunk, model, previous_summary)
        except Exception as e:
            logging.error(f"Fehler bei der Verarbeitung des Abschnitts: {e}")

    # Finalize the complete biography
    final_prompt = (
        "Hier ist die vollständige Biografie, die aus verschiedenen Abschnitten des Interviews extrahiert und kombiniert wurde. "
        "Bitte verfeinern Sie diese Biografie, indem Sie sie flüssiger und kohärenter machen, Redundanzen entfernen und sicherstellen, "
        "dass die Erzählung chronologisch und vollständig ist.\n\n{final_biography}"
    )

    # Print the final prompt
    print("\n==== Finale Eingabeaufforderung an das Modell ====\n")
    print(final_prompt.format(final_biography=previous_summary))

    final_biography = model.call(final_prompt.format(final_biography=previous_summary))

    # Print the final biography
    print("\n==== Endergebnis (Abschließende Biografie) ====\n")
    print(final_biography)

# Update the file path to the location of your pickle file
pickle_file_path = '/content/TranskriptOnly.pkl'
main(pickle_file_path)

<ipython-input-6-e002190390f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Transkript'] = chunk['Transkript'].astype(str)
<ipython-input-4-fce8cb8cd021>:17: DeprecationWarning: Call to deprecated function create.
  response = together.Completion.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:68: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)



==== Nächste Eingabeaufforderung an das Modell ====

Erstellen Sie eine vollständige und detaillierte Biografie basierend auf den folgenden Interviewinformationen. Konzentrieren Sie sich dabei auf wichtige Lebensereignisse und die in den Interviews erwähnten Jahre. Vermeiden Sie Wiederholungen und irrelevante Details.

Können wir anfangen? Also wäre schön, wenn Sie mit Kindheit beginnen würden. Ich war das erste Enkelkind, einzige Enkelkind, lange Zeit und bin sehr verwöhnt worden, da ich ziemlich viel bei der Großmutter gewesen bin. Ich bin gebürtig aus Hemer im Sauerland und bin 29.5.25 geboren. Und, ja, Kindheit verlief eigentlich an und für sich normal, habe allerdings ziemlich unter Migräne zu leiden gehabt schon als Vorschulkind, was sich während der Schulzeit verschlechterte und verschlimmerte. Ich hätte gerne die höhere Schule besucht, durfte aber nicht, weil mir immer gesagt wurde, ich sei krank. Die Schulentlassung, also den Hauptschulabschluss, habe ich gemacht 1939. Und da

<ipython-input-6-e002190390f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Transkript'] = chunk['Transkript'].astype(str)
<ipython-input-4-fce8cb8cd021>:17: DeprecationWarning: Call to deprecated function create.
  response = together.Completion.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:68: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)



==== Zwischenergebnis (Zusammenfassung des Abschnitts) ====

Und ich hab mich dann immer gefragt, wie soll ich damit auskommen, und ich hab mich einfach durchgeschlagen. Und dann habe ich wieder geheiratet. Das war im November 50. Und dann haben wir noch zwei Kinder. Und mein zweiter Mann ist dann 66 gestorben. Und dann bin ich alleine. Ich habe also insgesamt 5 Kinder. [vermischt] Fünf Kinder. Und da standen Sie jetzt alleine da. Und ich habe dann, ne, so vorhin habe ich Ihnen gesagt, dass ich so ein großes Glück habe, dass ich so einen umfassenden Freundeskreis habe. Und die helfen mir. Und ich habe auch noch eine Schwester, die noch lebt. Und die wohnt in der Nähe. Und ich habe noch eine Nichte, die auch noch lebt. Und die wohnt in der Nähe und die helfen mir auch alle beide. Und da bin ich ja jetzt auch noch beim, beim ..., eh, in der Kirchenfeier hier in der Kirchengemeinde. Und ich habe da noch mehrere Bekannte, die helfen mir, die mich fahren. Ich habe ja kein Auto mehr. Ich ha

<ipython-input-6-e002190390f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Transkript'] = chunk['Transkript'].astype(str)
<ipython-input-4-fce8cb8cd021>:17: DeprecationWarning: Call to deprecated function create.
  response = together.Completion.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:68: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)



==== Zwischenergebnis (Zusammenfassung des Abschnitts) ====

nte ich das. Und das war ja kein Geld. Aus den Verhältnissen, wenn ich also so groß geworden bin, voll, im Vollen, und dann den Unterschied gesehen habe, wie es vielen anderen ging. Ja, da hab ich mich halt auch so entwickelt. Ich hab mich halt so entwickelt, dass ich halt auch versucht habe, wo ich nur konnte, ... zu helfen, ne. Und das wird man nicht mehr los, ne. Ich mein, ich hab mich halt so entwickelt, dass ich halt aus meinem Leben etwas machen wollte. Und ich habe halt aus meinem Leben etwas gemacht. Ich hab halt aus meinem Leben was gemacht. Ich bin halt froh. Ich bin halt froh. Ich bin halt froh, dass ich so ein großes Glück habe, dass ich so einen umfassenden Freundeskreis habe. Ja, und ich habe ja auch noch eine Schwester, die noch lebt. Und die wohnt in der Nähe. Und ich habe noch eine Nichte, die auch noch lebt. Und die wohnt in der Nähe und die helfen mir auch alle beide. Und da bin ich ja jetzt auch noch beim